In [9]:
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from google.colab import drive
from tensorflow.keras.utils  import plot_model
from keras.layers import Dense, Dropout
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import class_weight
import numpy as np
import pandas as pd
import os

In [2]:
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_set = []
test_set = []
#print(os.listdir('/content/drive/My Drive/UC/Data3'))
for file in os.listdir('/content/drive/My Drive/UC/Data3/Train'):
        path = os.path.join('/content/drive/My Drive/UC/Data3/Train',file)
        df = pd.read_csv(path,index_col=None,header = 0)
        train_set.append(df)
for file in os.listdir('/content/drive/My Drive/UC/Data3/Test'):
        path = os.path.join('/content/drive/My Drive/UC/Data3/Test',file)
        df = pd.read_csv(path,index_col=None,header = 0)
        test_set.append(df)
        
df_train = pd.concat(train_set,axis=0,ignore_index=True)
df_test = pd.concat(test_set,axis=0,ignore_index=True)

df_all = pd.concat([df_train,df_test],axis=0,ignore_index=True)
train_shape = df_train.shape


(5749, 22)
(390, 22)
(6139, 22)
5749


In [4]:
smallest = df_all.nsmallest(int(0.7*len(df_all)),['Airbnb penetration']).index
largest = df_all.nlargest(int(0.05*len(df_all)),['Airbnb penetration']).index
#largest = df_all.nlargest(int(0.1*len(df_all)),['Airbnb penetration']).index

df_all['Airbnb penetration_class'] = [1]*len(df_all)
for i in smallest:
    df_all['Airbnb penetration_class'][i] = 0

for i in largest:
    df_all['Airbnb penetration_class'][i] = 2


<ipython-input-4-2ac6ef6fa267>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['Airbnb penetration_class'][i] = 0
<ipython-input-4-2ac6ef6fa267>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['Airbnb penetration_class'][i] = 2


In [5]:
df_all['Number of Hotels'] = df_all['Number of Hotels']**(1/2)
#df_all['Population Density'] = df_all['Population Density']**(1/2)
#df_all['Proportion of Owner Occupied Residences'] = df_all['Proportion of Owner Occupied Residences']**(1/2)
#df_all['Bus Stops'] = df_all['Bus Stops']**(1/2)
df_all['Point of Interests'] = df_all['Point of Interests']**(1/2)
df_all['Airbnb penetration'] = df_all['Airbnb penetration']**(1/2)

#Paper sets
set0 = ['Population Density','Bohemian Index', \
        'Talent Index','Proportion of Young People' \
        ,'Median Household Income','Median Household Value' \
        ,'Point of Interests','Distance to Center']

set1 = ['Population Density','Bohemian Index','Number of Hotels','Bus Stops','Distance to Center']


X_train = df_all[set1][:train_shape[0]]
    
y_train = df_all['Airbnb penetration_class'][:train_shape[0]]

X_test = df_all[set1][train_shape[0]:]
    
y_test = df_all['Airbnb penetration_class'][train_shape[0]:]

X_train = ((X_train-X_train.mean())/X_train.std())
X_test = ((X_test-X_test.mean())/X_test.std())

ROS = RandomOverSampler()
RUS = RandomUnderSampler()
SM = SMOTE()

X_train, y_train = SM.fit_resample(X_train,y_train)
#X_train, y_train = RUS.fit_resample(X_train,y_train)

y_train = to_categorical(y_train,num_classes=3)



In [6]:
def define_model(input_set):
  model = Sequential()
  model.add(Dense(8,input_dim=len(input_set),activation='tanh',kernel_initializer='he_uniform'))
  model.add(Dropout(0.2))
  model.add(Dense(5,activation='tanh',kernel_initializer='he_uniform'))
  model.add(Dropout(0.2))
  model.add(Dense(3,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  return model

In [ ]:
def evaluate(cm):
    #Low
    c_low = sum(cm[1]+cm[2])/sum(cm[0])
    print(c_low)
    TP_low = cm[0][0]
    FP_low = cm[1][0] + cm[2][0]
    FN_low = cm[0][1] + cm[0][2] 
    TN_low = cm[1][1] + cm[2][2] + cm[1][2] + cm[2][1]

    WA_low = ((c_low*TP_low)+TN_low)/((c_low*(TP_low + FN_low))+(TN_low + FP_low))
    print(WA_low)
    
    #Medium
    c_med = sum(cm[0]+cm[2])/sum(cm[1])
    TP_med = cm[1][1]
    FP_med = cm[0][1] + cm[2][1]
    FN_med = cm[1][0] + cm[1][2] 
    TN_med = cm[0][0] + cm[2][2] + cm[0][2] + cm[2][0]
    
    WA_med = ((c_med*TP_med)+TN_med)/((c_med*(TP_med + FN_med))+(TN_med + FP_med))
    print(WA_med)
    #High
    
    c_high = sum(cm[0]+cm[1])/sum(cm[2])
    TP_high = cm[2][2]
    FP_high = cm[0][2] + cm[1][2]
    FN_high = cm[2][0] + cm[2][1] 
    TN_high = cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0]

    WA_high = ((c_high*TP_high)+TN_high)/((c_high*(TP_high + FN_high))+(TN_high + FP_high))
    print(WA_high)
    
    return WA_low, WA_med, WA_high

In [ ]:
model = define_model(set1)
weights = {0:1,1:1,2:1}
history = model.fit(X_train, y_train, class_weight=weights, epochs=1000)

y_pred = model.predict(X_test)


Epoch 1/1000
378/378 [==============================] - 1s 2ms/step - loss: 1.0870
Epoch 2/1000
378/378 [==============================] - 1s 2ms/step - loss: 1.0001
Epoch 3/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9830
Epoch 4/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9703
Epoch 5/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9627
Epoch 6/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9602
Epoch 7/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9563
Epoch 8/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9528
Epoch 9/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9520
Epoch 10/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9450
Epoch 11/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9446
Epoch 12/1000
378/378 [==============================] - 1s 2ms/step - loss: 0.9450
E

In [ ]:
y_true = []
for i in y_pred:
  y_true.append(np.argmax(i))
cm = confusion_matrix(y_test, y_true)
l,m,h = evaluate(cm)
print("Accuracy:", (l+m+h)/3)

0.44981412639405205
0.6525699714276936
0.5777834464895413
0.6114683014354068
Accuracy: 0.6139405731175472
